In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import numpy as np
from src.features.seeds import SeedFeatures
from src.utils import load_data_template

In [81]:
data = load_data_template(season=False)
data.dropna(subset=['a_win'], inplace=True)
seed_feat = SeedFeatures(default_lags=0)
data = seed_feat.per_team_wrapper(data, seed_feat.team_seeds, per_game=False, per_day=False)
data.dropna(inplace=True)
print(data.shape)
data.sample()

(2117, 9)


,Season,team_a,team_b,in_target,game_set,a_win,DayNum,seed_team_a,seed_team_b
2078,2014,1372,1433,True,0,True,137,12,5


In [85]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data.drop(['Season', 'team_a', 'team_b', 'a_win', 'in_target', 'DayNum'], axis=1)
y = data[['a_win']].astype(int)

kf = KFold(n_splits=5, shuffle=True)
metrics = []
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    xgb = XGBClassifier()
    xgb.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = xgb.predict_proba(X_t.values)
    metric = log_loss(y_t.values, preds)
    print(metric)
    metrics.append(metric)
    
print('Metric Mean: {:.2f} ({:.2f})'.format(np.mean(metrics), np.std(metrics)))

0.5077694526671449
0.5697278088257419
0.5223429242069734
0.5505624859743525
0.5673358675013197
Metric Mean: 0.54 (0.02)
